In [ ]:
import pandas as pd

In [ ]:
path = "/content/Indicadores_municipales_sabana_DA.csv"

df = pd.read_csv(path, sep=",", encoding='ISO-8859-1')

In [ ]:
df.head(5)

,ent,nom_ent,mun,clave_mun,nom_mun,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,1,Aguascalientes,1,1001,Aguascalientes,794304,30.531104,2.264478,28.266627,27.983320,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,1,Aguascalientes,2,1002,Asientos,48592,67.111172,8.040704,59.070468,22.439389,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,1,Aguascalientes,3,1003,Calvillo,53104,61.360527,7.241238,54.119289,29.428583,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,1,Aguascalientes,4,1004,Cosío,14101,52.800458,4.769001,48.031458,27.128568,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,1,Aguascalientes,5,1005,Jesús María,101379,45.338512,6.084037,39.254475,26.262912,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083


**DATA CORRECTION**

In [ ]:
df = pd.read_csv('/content/Indicadores_municipales_sabana_DA.csv', encoding='ISO-8859-1')

df.drop(df.columns[:5], axis=1, inplace=True)

missing_values_before = df.isnull().sum().sum()
print(f"Total missing values before filling: {missing_values_before}")

for column in df.select_dtypes(include=['float64', 'int64']):
    df[column].fillna(df[column].median(), inplace=True)

for column in df.select_dtypes(include=['object']):
    df[column].fillna(df[column].mode()[0], inplace=True)

missing_values_after = df.isnull().sum().sum()
print(f"Total missing values after filling: {missing_values_after}")

df.to_csv('/content/cleaned_dataset.csv', index=False)

print("Cleaned dataset saved to '/content/cleaned_dataset.csv'")


Total missing values before filling: 305
Total missing values after filling: 0
Cleaned dataset saved to '/content/cleaned_dataset.csv'


**KNN**

In [93]:
import numpy as np
import pandas as pd

file_path = '/content/cleaned_dataset.csv'
df = pd.read_csv(file_path)

selected_features = ['pobreza', 'ic_rezedu', 'ic_asalud', 'ic_segsoc']
target_variable = 'porc_pob15_basicainc10'

selected_data = df[selected_features + [target_variable]].copy()
selected_data[target_variable] = (selected_data[target_variable] > 60).astype(int)

X = selected_data[selected_features].to_numpy()
y = selected_data[target_variable].to_numpy()

class KNNClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_new):
        self.distances = np.sqrt(np.sum((self.X_train - X_new)**2, axis=1))
        self.k_indices = np.argsort(self.distances)[:self.k]
        self.k_labels = self.y_train[self.k_indices]
        unique_labels, counts = np.unique(self.k_labels, return_counts=True)
        majority_vote = unique_labels[np.argmax(counts)]
        return majority_vote

    def calculate_accuracy(self, X_test, y_test):
        correct_predictions = 0
        total_predictions = len(y_test)

        for i in range(total_predictions):
            prediction = self.predict(X_test[i])
            if prediction == y_test[i]:
                correct_predictions += 1

        accuracy = correct_predictions / total_predictions
        return accuracy

shuffled_indices = np.random.permutation(len(X))
train_indices = shuffled_indices[:int(0.8 * len(X))]
test_indices = shuffled_indices[int(0.8 * len(X)):]

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

knn = KNNClassifier(k=3)
knn.fit(X_train, y_train)

accuracy_method = knn.calculate_accuracy(X_test, y_test)

predictions = np.array([knn.predict(x) for x in X_test])
correct_predictions = np.sum(predictions == y_test)
total_predictions = len(y_test)
accuracy_math = correct_predictions / total_predictions

print(f"Accuracy (using method): {accuracy_method*100:.2f}%")
print(f"Accuracy (calculated mathematically): {accuracy_math*100:.2f}%")

random_index = np.random.randint(0, len(X_test))
X_new = X_test[random_index]

majority_vote = knn.predict(X_new)

print("Random Data Point: ", X_new)
print("Distances: ", knn.distances)
print("Indices of 3 closest points: ", knn.k_indices)

if majority_vote == 1:
    print("The random point is predicted to have a High probability of not finishing high school.")
else:
    print("The random point is predicted to have a Low probability of not finishing high school.")


Accuracy (using method): 85.57%
Accuracy (calculated mathematically): 85.57%
Random Data Point:  [26.57807617 12.597288   22.20176336 36.08728231]
Distances:  [27.82431555 52.0946087  78.65545754 ... 38.99789364 88.35562899
 80.11886054]
Indices of 3 closest points:  [1225 1882   57]
The random point is predicted to have a Low probability of not finishing high school.


**KNN LIBRARIES**

In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

file_path = '/content/cleaned_dataset.csv'
df = pd.read_csv(file_path)

selected_features = ['pobreza', 'ic_rezedu', 'ic_asalud', 'ic_segsoc']
target_variable = 'porc_pob15_basicainc10'

selected_data = df[selected_features + [target_variable]].copy()
selected_data[target_variable] = (selected_data[target_variable] > 60).astype(int)

X = selected_data[selected_features].values
y = selected_data[target_variable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

predictions = knn.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy using sklearn's KNN: {accuracy*100:.2f}%")

random_index = np.random.randint(0, len(X_test))
X_new = X_test[random_index].reshape(1, -1)

majority_vote = knn.predict(X_new)

print("\nRandom Data Point: ", X_new)
if majority_vote == 1:
    print("The random point is predicted to have a High probability of not finishing high school.")
else:
    print("The random point is predicted to have a Low probability of not finishing high school.")


Accuracy using sklearn's KNN: 86.59%

Random Data Point:  [[57.4442265  26.07354689 16.34353282 62.76893197]]
The random point is predicted to have a Low probability of not finishing high school.


**PERCEPTRON**

In [76]:
import numpy as np
import pandas as pd

file_path = '/content/cleaned_dataset.csv'
df = pd.read_csv(file_path)

selected_features = ['pobreza', 'ic_rezedu', 'ic_asalud', 'ic_segsoc']
target_variable = 'porc_pob15_basicainc10'

selected_data = df[selected_features + [target_variable]].copy()
selected_data[target_variable] = (selected_data[target_variable] > 60).astype(int)

X = selected_data[selected_features].to_numpy()
y = selected_data[target_variable].to_numpy()

class Perceptron:
    def __init__(self, input_size, lr=0.1, epochs=100):
        self.weights = np.zeros(input_size+1)
        self.lr = lr
        self.epochs = epochs

    def activation(self, x):
        return 1 if x >= 0 else 0

    def predict(self, x):
        z = self.weights.T.dot(x)
        return self.activation(z)

    def fit(self, X, y):
        for _ in range(self.epochs):
            for i in range(y.shape[0]):
                x = np.insert(X[i], 0, 1)
                predicted = self.predict(x)
                error = y[i] - predicted
                self.weights = self.weights + self.lr * error * x

    def calculate_accuracy(self, X, y):
        correct_predictions = 0
        total_predictions = X.shape[0]
        for i in range(total_predictions):
            x = np.insert(X[i], 0, 1)
            if self.predict(x) == y[i]:
                correct_predictions += 1
        return correct_predictions / total_predictions

shuffled_indices = np.random.permutation(len(X))
train_indices = shuffled_indices[:int(0.8 * len(X))]
test_indices = shuffled_indices[int(0.8 * len(X)):]

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

perceptron = Perceptron(input_size=len(selected_features))
perceptron.fit(X_train, y_train)

accuracy_method = perceptron.calculate_accuracy(X_test, y_test)

predictions = np.array([perceptron.predict(np.insert(x, 0, 1)) for x in X_test])
correct_predictions = np.sum(predictions == y_test)
total_predictions = len(y_test)
accuracy_math = correct_predictions / total_predictions

print(f"Accuracy (using method): {accuracy_method*100:.2f}%")
print(f"Accuracy (calculated mathematically): {accuracy_math*100:.2f}%")

random_index = np.random.randint(0, len(X_test))
X_new = np.insert(X_test[random_index], 0, 1)

prediction = perceptron.predict(X_new)

if prediction == 1:
    print("The random point is predicted to have a High probability of not finishing high school.")
else:
    print("The random point is predicted to have a Low probability of not finishing high school.")


Accuracy (using method): 71.14%
Accuracy (calculated mathematically): 71.14%
The random point is predicted to have a Low probability of not finishing high school.


**PERCEPTRON LIBRARIERS**

In [95]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

file_path = '/content/cleaned_dataset.csv'
df = pd.read_csv(file_path)

selected_features = ['pobreza', 'ic_rezedu', 'ic_asalud', 'ic_segsoc']
target_variable = 'porc_pob15_basicainc10'

selected_data = df[selected_features + [target_variable]].copy()
selected_data[target_variable] = (selected_data[target_variable] > 60).astype(int)

X = selected_data[selected_features].values
y = selected_data[target_variable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

perceptron = Perceptron(max_iter=100, eta0=0.1)
perceptron.fit(X_train, y_train)

predictions = perceptron.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy using sklearn's Perceptron: {accuracy*100:.2f}%")

random_index = np.random.randint(0, len(X_test))
X_new = X_test[random_index].reshape(1, -1)

prediction = perceptron.predict(X_new)

print("\nRandom Data Point: ", X_new)
if prediction == 1:
    print("The random point is predicted to have a High probability of not finishing high school.")
else:
    print("The random point is predicted to have a Low probability of not finishing high school.")


Accuracy using sklearn's Perceptron: 64.84%

Random Data Point:  [[23.13898004 12.33889451 31.79335372 45.0568469 ]]
The random point is predicted to have a Low probability of not finishing high school.
